## 生成选题文档-提示词

```markdown
### input-output
- Folder："`D:\Github\lianxhta\Python`"
- filename-input：包含关键词 '选题' 且后缀为 '.md' 的文件
- range-转换范围: 
  - begin: # B784
  - end: # B802
- Folder-output：'.\topics'

### 任务：

- Input: 读取 {filename-input} 文件。
- Actions：
  - 在 {filename-input} 文档中，以 `^# B\d{3,.}：title` 开头的行为分割线，将该行及以下到下一个 `^# B\d{3,.}：title` 之间的文本写入一个新的 **.md** 文件中
  - 仅读取 {begin} 到 {end} 范围内的选题
- Output:
  - 存入：{Folder-output} 文件夹
    - 预先清空该文件夹 
  - 文件名为 `B\d{3,.}：title` 
    - 首字母为 'B'
    - 文件后缀为 '.md'，而不是 '.md.md'
  - 屏幕打印：
    - 文件总数
    - 文件名称列表
```


In [7]:
import os
import re
import shutil

# ================= 用户参数 =================
input_folder = r"D:\Github\lianxhta\Python"
output_folder = r".\topics"
begin_tag = "# B802"
end_tag = "# B784"

# ================= 辅助函数 =================
def get_input_filename(folder):
    for fname in os.listdir(folder):
        if "选题" in fname and fname.endswith('.md'):
            return os.path.join(folder, fname)
    raise FileNotFoundError("未找到包含 '选题' 且后缀为 '.md' 的文件")

def read_file(filepath):
    with open(filepath, encoding="utf-8") as f:
        return f.read()

def split_by_title(text):
    # 正则匹配 '# Bxxx：title'
    blocks = list(re.finditer(r'^# B\d{3,}：.*$', text, flags=re.MULTILINE))
    result = []
    for idx, match in enumerate(blocks):
        start = match.start()
        end = blocks[idx + 1].start() if idx + 1 < len(blocks) else len(text)
        title_line = text[match.start():match.end()]
        block_text = text[start:end].strip()
        result.append((title_line, block_text))
    return result

def find_range(blocks, begin_tag, end_tag):
    begin_idx, end_idx = None, None
    for i, (title, _) in enumerate(blocks):
        if title.strip().startswith(begin_tag):
            begin_idx = i
        if title.strip().startswith(end_tag):
            end_idx = i
    if begin_idx is not None and end_idx is not None and begin_idx <= end_idx:
        return blocks[begin_idx:end_idx+1]
    else:
        raise ValueError("未能定位 begin 或 end 位置，或顺序错误")

def clean_folder(folder):
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)

def save_blocks_to_folder(blocks, folder):
    filenames = []
    for title, content in blocks:
        # 文件名采用 '# Bxxx：title' 的形式
        file_base = re.match(r'^# (B\d{3,}：.*)$', title.strip())
        if not file_base:
            continue
        filename = file_base.group(1) + ".md"
        # 替换非法字符
        filename = re.sub(r'[\\/:*?"<>|]', '_', filename)
        filepath = os.path.join(folder, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content.strip() + "\n")
        filenames.append(filename)
    return filenames

# ================= 主程序 =================
if __name__ == "__main__":
    # 1. 获取输入文件
    input_path = get_input_filename(input_folder)
    # 2. 读取内容
    text = read_file(input_path)
    # 3. 按分割线分割
    blocks = split_by_title(text)
    # 4. 提取 begin 到 end 范围
    blocks_in_range = find_range(blocks, begin_tag, end_tag)
    # 5. 预清空输出文件夹
    clean_folder(output_folder)
    # 6. 写入输出
    filelist = save_blocks_to_folder(blocks_in_range, output_folder)
    # 7. 打印信息
    print(f"文件总数：{len(filelist)}")
    print("文件名称列表：")
    for fn in filelist:
        print(fn)


文件总数：19
文件名称列表：
B802：Python 仓库介绍：skfolio.md
B801：新书推介：大语言模型精要.md
B800：翻译：LLM-Agents简介.md
B799：翻译：词向量的原理.md
B798：bnlearn-基于贝叶斯的因果关系自动发掘工具.md
B797：Python包：distfit-分布拟合.md
B796：marker：快速将PDF表格转换为Markdown或JSON数据.md
B795：推文+小视频：英文PDF文档转中文.md
B794：公开API资源汇总.md
B793：Stata：biastest-不同模型间系数差异检验.md
B792：Python 包介绍：akshare.md
B791：数据分析常用提示词梳理.md
B790：如何组织AI提示词工作流程.md
B789：编程助手：Windsurf Plugin.md
B788：Python：期权定价公式及实现.md
B787：ylearn：轻松实现因果分析的Python包.md
B786：HyperTS：端到端的时间序列分析工具.md
B785：FinanceDatabase：Github公开数据仓库简介.md
B784：论文推介：贝叶斯SFA.md
